Введение

Метрические методы основаны на гипотезе компактности, суть которой состоит в том, что объекты с похожими признаковыми описаниями имеют похожие значения целевой переменной. Если эта гипотеза верна, то строить прогноз для нового объекта можно на основе близких к нему объектов из обучающей выборки — например, путем усреднения их ответов (для регрессии) или путем выбора наиболее популярного среди них класса (для классификации). Методы такого типа и называются метрическими. Они имеют несколько особенностей:

Процедура обучения, по сути, отсутствует — достаточно лишь запомнить все объекты обучающей выборки
Можно использовать метрику, учитывающую особенности конкретного набора данных — например, наличие категориальных (номинальных) признаков
При правильном выборе метрики и достаточном размере обучающей выборки метрические алгоритмы показывают качество, близкое к оптимальному
Метрические методы чувствительны к масштабу признаков — так, если масштаб одного из признаков существенно превосходит масштабы остальных признаков, то их значения практически не будут влиять на ответы алгоритма. Поэтому важно производить масштабирование признаков. Обычно это делается путем вычитания среднего значения признака и деления на стандартное отклонение.

In [1]:
import pandas as pd
import numpy as np

In [2]:
url ='https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'
data = pd.read_csv(url, header=None)
data.tail(6)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
172,3,14.16,2.51,2.48,20.0,91,1.68,0.70,0.44,1.24,9.7,0.62,1.71,660
173,3,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.7,0.64,1.74,740
174,3,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.3,0.70,1.56,750
175,3,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.2,0.59,1.56,835
176,3,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.3,0.60,1.62,840
177,3,14.13,4.10,2.74,24.5,96,2.05,0.76,0.56,1.35,9.2,0.61,1.60,560


Реализация в Scikit-Learn

Метод k ближайших соседей реализован в классе sklearn.neighbors.KNeighborsClassifier. 
Основным параметром является n_neighbors, который задает число соседей для построения прогноза.

Вам понадобится производить кросс-валидацию по блокам. Кросс-валидация заключается в разделении 
выборки на m непересекающихся блоков примерно одинакового размера, после чего выполняется m шагов. 
На i-м шаге i-й блок выступает в качестве тестовой выборки, объединение всех остальных блоков — в качестве обучающей выборки.
Соответственно, на каждом шаге алгоритм обучается на некоторо
й обучающей выборке, после чего вычисляется его качество на тестовой выборке.
После выполнение m шагов мы получаем m показателей к
ачества, усреднение которых и дает оценку кросс-валидации. 
Подробнее вы можете послушать про кросс-валидацию в видео "Проблема переобучения. Методология решения задач машинного обучения" 
из первого модуля, а также почитать на Википедии (на русском или на английском) или в документации scikit-learn.
Технически кросс-валидация проводится в два этапа:

Создается генератор разбиений sklearn.cross_validation.KFold, который задает набор разбиений на обучение и валидацию.
Число блоков в кросс-валидации определяется параметром n_folds. Обратите внимание, что порядок следования объектов в выборке
может быть неслучайным, это может привести к смещенности кросс-валидационной оценки. Чтобы устранить такой эффект, объекты
выборки случайно перемешивают перед разбиением на блоки. Для перемешивания достаточно передать генератору KFold параметр
shuffle=True.
Вычисляется качество на всех разбиениях можно при помощи функции sklearn.cross_validation.cross_val_score. В качестве 
параметра estimator передается классификатор, в качестве параметра cv — генератор разбиений с предыдущего шага. С помощью
параметра scoring можно задавать меру качества, по умолчанию в задачах классификации используется доля верных ответов (accuracy). 
Результатом является массив, значения которого нужно усреднить.
Приведение признаков к одному масштабу можно делать с помощью функции sklearn.preprocessing.scale, которой на вход 
необходимо подать матрицу признаков и получить масштабированную матрицу, в которой каждый столбец имеет нулевое 
среднее значение и единичное стандартное отклонение.

In [7]:
data.shape

(178, 14)

In [9]:
data.count(axis=0)

0     178
1     178
2     178
3     178
4     178
5     178
6     178
7     178
8     178
9     178
10    178
11    178
12    178
13    178
dtype: int64

In [3]:
X = data.drop((0), axis=1)  # Выбрасываем столбец 'class'.
y = data[0]
X.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [94]:
from sklearn.neighbors import KNeighborsClassifier

In [95]:
knn=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')
knn.fit(X,y)
from sklearn.cross_validation import KFold,cross_val_score
kf=KFold(n=data.shape[0],n_folds=5,shuffle=True,random_state=42)
scores =cross_val_score(knn, X, y, cv=kf,
 n_jobs=1)
print(np.mean(scores))

0.730476190476


In [96]:
from sklearn.cross_validation import KFold,cross_val_score
for i in range(1,51):
    knn=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=i, p=2,
           weights='uniform')
    knn.fit(X,y)
    kf=KFold(n=data.shape[0],n_folds=5,shuffle=True,random_state=42)
    scores =cross_val_score(knn, X, y, cv=kf,
     n_jobs=1)
    print(i,np.mean(scores))

1 0.730476190476
2 0.66253968254
3 0.708253968254
4 0.657777777778
5 0.674603174603
6 0.674285714286
7 0.68
8 0.68
9 0.702380952381
10 0.680158730159
11 0.70253968254
12 0.696666666667
13 0.690952380952
14 0.679365079365
15 0.701904761905
16 0.679523809524
17 0.701587301587
18 0.679523809524
19 0.679365079365
20 0.690952380952
21 0.701904761905
22 0.696507936508
23 0.702063492063
24 0.707619047619
25 0.701904761905
26 0.696507936508
27 0.696349206349
28 0.707936507937
29 0.713492063492
30 0.707936507937
31 0.690952380952
32 0.713492063492
33 0.713492063492
34 0.724603174603
35 0.724603174603
36 0.713492063492
37 0.713492063492
38 0.713492063492
39 0.707936507937
40 0.707777777778
41 0.713492063492
42 0.707777777778
43 0.707777777778
44 0.707777777778
45 0.707777777778
46 0.707777777778
47 0.696666666667
48 0.719047619048
49 0.696666666667
50 0.707777777778


In [4]:
from sklearn.preprocessing import scale
X_matr=np.matrix(X)
X_scale=scale(X_matr)
X_scale=pd.DataFrame(X_scale)
X_scale.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1.518613,-0.562250,0.232053,-1.169593,1.913905,0.808997,1.034819,-0.659563,1.224884,0.251717,0.362177,1.847920,1.013009
1,0.246290,-0.499413,-0.827996,-2.490847,0.018145,0.568648,0.733629,-0.820719,-0.544721,-0.293321,0.406051,1.113449,0.965242
2,0.196879,0.021231,1.109334,-0.268738,0.088358,0.808997,1.215533,-0.498407,2.135968,0.269020,0.318304,0.788587,1.395148
3,1.691550,-0.346811,0.487926,-0.809251,0.930918,2.491446,1.466525,-0.981875,1.032155,1.186068,-0.427544,1.184071,2.334574
4,0.295700,0.227694,1.840403,0.451946,1.281985,0.808997,0.663351,0.226796,0.401404,-0.319276,0.362177,0.449601,-0.037874


In [99]:
from sklearn.cross_validation import KFold,cross_val_score
lu4=-1
lu4_sosedi=0
for i in range(1,51):
    knn=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=i, p=2,
           weights='uniform')
    knn.fit(X_scale,y)
    kf=KFold(n=data.shape[0],n_folds=5,shuffle=True,random_state=42)
    scores =cross_val_score(knn, X_scale, y, cv=kf,
     n_jobs=1)
    sredn=np.mean(scores)
    sosedi=i
    if lu4<sredn:
        lu4=sredn
        lu4_sosedi=sosedi
print(lu4_sosedi,lu4)

29 0.977619047619


In [ ]:
Введение

Главным параметром любого метрического алгоритма является функция расстояния (или метрика), используемая для измерения сходства
между объектами. Можно использовать стандартный вариант (например, евклидову метрику), но гораздо более эффективным вариантом 
является подбор метрики под конкретную задачу. Один из подходов — использование той же евклидовой метрики, но с весами: каждой
координате ставится в соответствие определенный коэффициент; чем он больше, тем выше вклад признака в итоговое расстояние.
Веса настраиваются с целью оптимизации качества на отложенной выборке.
Другой подход, о котором и пойдет речь в данном задании — выбор метрики из некоторого класса метрик. 
Мы возьмем за основу метрику Минковского

In [1]:
from sklearn.datasets import load_boston
boston = load_boston()
X1=boston.data
y1=boston.target
X1_matr=np.matrix(X1)
X1_scale=sklearn.preprocessing.scale(X1_matr)
X1_scale=pd.DataFrame(X1_scale)
y1=pd.DataFrame(y1)
y1.head()

NameError: name 'np' is not defined

In [102]:
shag=np.linspace(1,10,200)
len(shag)

(506,)


In [108]:
from sklearn.neighbors import KNeighborsRegressor

In [118]:
from sklearn.cross_validation import KFold,cross_val_score
parametr=0
sr_oshibka=-1000
for i in range(len(shag)):
    knn=KNeighborsRegressor(n_jobs=1, n_neighbors=5, p=shag[i],weights='distance')
    kNN.fit(X1_scale,y1)
    kf=KFold(n=y1.shape[0],n_folds=5,shuffle=True,random_state=42)
    scores =cross_val_score(knn, X1_scale, y1, cv=kf,n_jobs=1,scoring='mean_squared_error')
    print(shag[i],np.mean(scores))

1.0 -16.0502085084
1.04522613065 -16.4040806015
1.09045226131 -16.3672292229
1.13567839196 -16.4425387869
1.18090452261 -16.45528062
1.22613065327 -16.5065632054
1.27135678392 -16.6167523382
1.31658291457 -16.8019119949
1.36180904523 -16.8439573766
1.40703517588 -17.0946106323
1.45226130653 -17.1200151074
1.49748743719 -16.9513500813
1.54271356784 -17.0889266772
1.58793969849 -16.7910804282
1.63316582915 -16.9708836013
1.6783919598 -17.0061639569
1.72361809045 -17.1542553046
1.76884422111 -17.1715946679
1.81407035176 -17.2001731656
1.85929648241 -17.3194042213
1.90452261307 -17.4108902784
1.94974874372 -17.382596583
1.99497487437 -17.3235384553
2.04020100503 -17.3645959299
2.08542713568 -17.3901598226
2.13065326633 -17.6635109941
2.17587939698 -17.5748798407
2.22110552764 -17.3276454948
2.26633165829 -17.5156418926
2.31155778894 -17.5654292129
2.3567839196 -17.5761891114
2.40201005025 -17.6206711305
2.4472361809 -17.6238009382
2.49246231156 -17.3068846192
2.53768844221 -17.3513103151
2